This is a notebook to try out the standard Voltronic RS232 protocol.

## Checksum

The Voltronic protocol uses CRC-16 (XMODEM variant) for calculating the checksum for messages.

In [1]:
CRC16_XMODEM_TABLE = [
    0x0000, 0x1021, 0x2042, 0x3063, 0x4084, 0x50a5, 0x60c6, 0x70e7,
    0x8108, 0x9129, 0xa14a, 0xb16b, 0xc18c, 0xd1ad, 0xe1ce, 0xf1ef,
    0x1231, 0x0210, 0x3273, 0x2252, 0x52b5, 0x4294, 0x72f7, 0x62d6,
    0x9339, 0x8318, 0xb37b, 0xa35a, 0xd3bd, 0xc39c, 0xf3ff, 0xe3de,
    0x2462, 0x3443, 0x0420, 0x1401, 0x64e6, 0x74c7, 0x44a4, 0x5485,
    0xa56a, 0xb54b, 0x8528, 0x9509, 0xe5ee, 0xf5cf, 0xc5ac, 0xd58d,
    0x3653, 0x2672, 0x1611, 0x0630, 0x76d7, 0x66f6, 0x5695, 0x46b4,
    0xb75b, 0xa77a, 0x9719, 0x8738, 0xf7df, 0xe7fe, 0xd79d, 0xc7bc,
    0x48c4, 0x58e5, 0x6886, 0x78a7, 0x0840, 0x1861, 0x2802, 0x3823,
    0xc9cc, 0xd9ed, 0xe98e, 0xf9af, 0x8948, 0x9969, 0xa90a, 0xb92b,
    0x5af5, 0x4ad4, 0x7ab7, 0x6a96, 0x1a71, 0x0a50, 0x3a33, 0x2a12,
    0xdbfd, 0xcbdc, 0xfbbf, 0xeb9e, 0x9b79, 0x8b58, 0xbb3b, 0xab1a,
    0x6ca6, 0x7c87, 0x4ce4, 0x5cc5, 0x2c22, 0x3c03, 0x0c60, 0x1c41,
    0xedae, 0xfd8f, 0xcdec, 0xddcd, 0xad2a, 0xbd0b, 0x8d68, 0x9d49,
    0x7e97, 0x6eb6, 0x5ed5, 0x4ef4, 0x3e13, 0x2e32, 0x1e51, 0x0e70,
    0xff9f, 0xefbe, 0xdfdd, 0xcffc, 0xbf1b, 0xaf3a, 0x9f59, 0x8f78,
    0x9188, 0x81a9, 0xb1ca, 0xa1eb, 0xd10c, 0xc12d, 0xf14e, 0xe16f,
    0x1080, 0x00a1, 0x30c2, 0x20e3, 0x5004, 0x4025, 0x7046, 0x6067,
    0x83b9, 0x9398, 0xa3fb, 0xb3da, 0xc33d, 0xd31c, 0xe37f, 0xf35e,
    0x02b1, 0x1290, 0x22f3, 0x32d2, 0x4235, 0x5214, 0x6277, 0x7256,
    0xb5ea, 0xa5cb, 0x95a8, 0x8589, 0xf56e, 0xe54f, 0xd52c, 0xc50d,
    0x34e2, 0x24c3, 0x14a0, 0x0481, 0x7466, 0x6447, 0x5424, 0x4405,
    0xa7db, 0xb7fa, 0x8799, 0x97b8, 0xe75f, 0xf77e, 0xc71d, 0xd73c,
    0x26d3, 0x36f2, 0x0691, 0x16b0, 0x6657, 0x7676, 0x4615, 0x5634,
    0xd94c, 0xc96d, 0xf90e, 0xe92f, 0x99c8, 0x89e9, 0xb98a, 0xa9ab,
    0x5844, 0x4865, 0x7806, 0x6827, 0x18c0, 0x08e1, 0x3882, 0x28a3,
    0xcb7d, 0xdb5c, 0xeb3f, 0xfb1e, 0x8bf9, 0x9bd8, 0xabbb, 0xbb9a,
    0x4a75, 0x5a54, 0x6a37, 0x7a16, 0x0af1, 0x1ad0, 0x2ab3, 0x3a92,
    0xfd2e, 0xed0f, 0xdd6c, 0xcd4d, 0xbdaa, 0xad8b, 0x9de8, 0x8dc9,
    0x7c26, 0x6c07, 0x5c64, 0x4c45, 0x3ca2, 0x2c83, 0x1ce0, 0x0cc1,
    0xef1f, 0xff3e, 0xcf5d, 0xdf7c, 0xaf9b, 0xbfba, 0x8fd9, 0x9ff8,
    0x6e17, 0x7e36, 0x4e55, 0x5e74, 0x2e93, 0x3eb2, 0x0ed1, 0x1ef0,
]

def crc16_xmodem(data: bytes) -> int:
    crc = 0

    for byte in data:
        index = ((crc >> 8) ^ byte) & 0xFF
        crc = (CRC16_XMODEM_TABLE[index] ^ (crc << 8)) & 0xFFFF

    return crc & 0xFFFF

In [2]:
hex(crc16_xmodem(bytes("QPI", "ascii")))
# Expected output: 0xbeac

'0xbeac'

In [3]:
def pack_message(message: str, encoding='ascii') -> bytes:
    """
    Packs the message into the transmission ready format.
    
    Calculates the checksum of the message and appends it to the message with a carriage return
    as specified by the protocol.
    """
    result = bytearray(message, encoding)

    checksum = crc16_xmodem(bytes(result))
    
    result.append(checksum >> 8)
    result.append(checksum & 0xFF)
    result.append(0x0D) # Carriage Return

    return bytes(result)

("QPI", pack_message("QPI").hex(' '))

('QPI', '51 50 49 be ac 0d')

In [4]:
def unpack_message(data: bytes, encoding='ascii') -> str:
    """
    Unpack the message from the transmitted format.

    Validates the checksum and strips it from the message as specified by the protocol.
    """

    if data[-1] != 0x0D:
        print(f'Data: {data.decode("ascii", errors="backslashreplace")}')
        raise Exception("Message missing carriage return.")
    raw_message = data[:-3]
    
    packed_checksum = (data[-3] << 8) | data[-2]
    calculated_checksum = crc16_xmodem(raw_message)

    if packed_checksum != calculated_checksum:
        print(f'Data: {data.decode("ascii", errors="backslashreplace")}')
        print(f'Calculate Checksum: {calculated_checksum}')
        print(f'   Packed Checksum: {packed_checksum}')
        raise Exception("Unmatching checksum.")
    
    if raw_message[0] == 0x28: # '(' character.
        raw_message = raw_message[1:]

    return raw_message.decode(encoding)

unpack_message(pack_message('QPI'))

'QPI'

In [7]:
pack_message('VOLTRONICS').hex(' ')

'56 4f 4c 54 52 4f 4e 49 43 53 6c 3e 0d'

In [8]:
pack_message('#$!@-,;:[]/').hex(' ')

'23 24 21 40 2d 2c 3b 3a 5b 5d 2f e1 a2 0d'